# How to use the error correction utility

An analysis based on a dataset of sources for which the available observations per source were randomly split in two sets thus leading to the generation of two mean spectra for each object, has shown that errors on synthetic photometry are somewhat underestimated. 

This same dataset has been used to compute empirical corrections to the uncertainties (see Montegriffo et al. 2022). These corrections are available in GaiaXPy and can be applied to a set of synthethic photometry.

## Basic usage

In [1]:
# Import the tool
from gaiaxpy import apply_error_correction

The error correction can be applied over a synthetic photometry DataFrame which can be generated using the generator tool.

In [2]:
# Import generator
from gaiaxpy import generate

A photometric system is required to generate synthetic photometry.

In [3]:
# Import photometric systems
from gaiaxpy import PhotometricSystem

We can see which systems are available.

In [4]:
PhotometricSystem.get_available_systems()

'Els_Custom_W09_S2, Euclid_VIS, Gaia_2, Gaia_DR3_Vega, Halpha_Custom_AB, H_Custom, Hipparcos_Tycho, HST_ACSWFC, HST_HUGS_Std, HST_WFC3UVIS, HST_WFPC2, IPHAS, JKC, JKC_Std, JPAS, JPLUS, JWST_NIRCAM, PanSTARRS1, PanSTARRS1_Std, Pristine, SDSS, SDSS_Std, Stromgren, Stromgren_Std, WFIRST'

In [5]:
# We can select a simple system
phot_system = PhotometricSystem.Gaia_DR3_Vega
# Or use a list of them
phot_systems_list = [PhotometricSystem.Pristine, PhotometricSystem.JWST_NIRCAM, PhotometricSystem.Euclid_VIS]

To generate synthetic photometry:

In [6]:
# File with input data
f = '/home/drm/GaiaXPy/tests/files/xp_continuous/XP_CONTINUOUS_RAW_dr3int6.ecsv'
synthetic_photometry = generate(f, photometric_system=phot_systems_list)
synthetic_photometry

source_id  Pristine_mag_CaHK  Pristine_flux_CaHK  Pristine_flux_error_CaHK  \
0          6          13.818329        2.162462e-16              9.483388e-18   
1          4          13.016468        4.525774e-16              4.957204e-18   

   JwstNircam_mag_F070W  JwstNircam_mag_F090W  JwstNircam_flux_F070W  \
0              8.649869              6.548173           5.939262e-15   
1             13.339628             13.556191           7.903671e-17   

   JwstNircam_flux_F090W  JwstNircam_flux_error_F070W  \
0           2.037843e-14                 7.313134e-18   
1           3.206001e-17                 5.804090e-20   

   JwstNircam_flux_error_F090W  EuclidVis_mag_VIS  EuclidVis_flux_VIS  \
0                 1.122291e-17           8.229913        8.775556e-15   
1                 5.476856e-20          13.318073        8.091163e-17   

   EuclidVis_flux_error_VIS  
0              3.765390e-18  
1              4.189795e-20

We can now apply the correction over this data:

Due to the fact that the error corrections are computed from the Gaia photometry, if the system Gaia_DR3_Vega is not present in the input photometry, an error will be raised. We need to pass a list that contains this system and recompute the photometry.

In [7]:
# Add the system
phot_systems_list = [PhotometricSystem.Pristine, PhotometricSystem.JWST_NIRCAM, PhotometricSystem.Euclid_VIS, PhotometricSystem.Gaia_DR3_Vega]
# Recompute
synthetic_photometry = generate(f, photometric_system=phot_systems_list)
# Now we can apply the error correction
synth_phot_corrected = apply_error_correction(synthetic_photometry, photometric_system=phot_systems_list)
# Not every system has a correction table. A warning will be raised for every system for which a correction table does not exist.

If the photometric systems are not passed as an argument, the program will infer them from the data:

In [8]:
synth_phot_corrected_infer = apply_error_correction(synthetic_photometry)

We can verify that both outputs are equivalent:

In [9]:
print(f'Are both frames equal? {synth_phot_corrected.equals(synth_phot_corrected_infer)}')

Are both frames equal? True


## Advanced usage

Additional arguments can be passed to the function.

These are:
1. output_path
2. output_file
3. output_format
4. save_file

Three parameters: **output_path**, **output_file**, and **output_format** define the entire path of the resulting file.

The default output path is the current path. If the given output path does not exist, it will be created.

The default output file name is 'output_spectra'. 

The default output format is the format of the input file (i.e. if the input file is a 'fits', then the output file will be a FITS file by default.), or CSV in any other case (DataFrame, ADQL query or list).

**NOTE: If a file with the same path and name already exists, it will be AUTOMATICALLY OVERWRITTEN.**

In [10]:
synth_phot_corrected = apply_error_correction(synthetic_photometry, output_path='/home/drm/Desktop', output_file='my_file', output_format='csv')

The additional parameter **save_file** is a boolean that tells the program whether to save the results or not.
If 'output_file' is given but 'save_file' is set to False, a warning will be raised.

In [11]:
synth_phot_corrected = apply_error_correction(synthetic_photometry, output_file='my_file', save_file=False)